## Needed Modules

In [2]:
import pandas as pd
import json
import time
import numpy as np
from tqdm import tqdm
from typing import *
import re
import os
from IPython.display import clear_output
import shutil

# Scrapping and crawling modules
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
import warnings


# New chromediver
import undetected_chromedriver as uc
from requests.utils import quote, unquote
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException

In [ ]:
def getTime(str):
    '''
    This function is used to convert string of datetime in isoformat to datetime object
    Params:
        str:
            - String of datetime in isoformat
    return:
        date_time_obj:
            - Datetime object
    '''
    date_time_obj = datetime.fromisoformat(str)
    date_time_obj =date_time_obj + timedelta(hours=7)
    return date_time_obj

def minOneDay(str):
    '''
    This function is used to subtract one day from the given date
    Params:
        str:
            - String of datetime in isoformat
    return:
        str:
            - String of datetime in isoformat after subtracting one day
    '''
    str = datetime.strptime(str, "%Y-%m-%d")
    str = str - timedelta(days=1)
    str = str.strftime("%Y-%m-%d")
    return str

def wait(timeout: int = 10):
    '''
    just a glorified simple function to wait for a certain amount of time
    '''
    for i in tqdm(range(timeout), desc="Waiting"):
        time.sleep(1)
    clear_output()


def safe_int_from_aria(aria_label: str) -> int:
    match = re.search(r"\d+", aria_label or "")
    return int(match.group(0)) if match else 0

class twitterScrapper:
    def __init__(self, credentials: str = "Credentials/twitter.json",
                 ):
        '''
        This function is used to initialize the class and will also login to twitter
        
        Parameters
        ----------
        credentials : str
            - Path to the twitter credentials json file
            - The json file should be in the following format:
            ```
            {   "username" : "your_username",
                "password" : "your_password",
                "email"    : "your_email"}
            ```
        '''

        if not os.path.exists(credentials):
            raise FileNotFoundError("Credentials file not found!")

        with open(credentials, "r") as f:
            credentials = json.load(f)
            try:
                username = credentials["username"]
                password = credentials["password"]
                email = credentials["email"]
            except KeyError:
                raise KeyError("Credentials file is not in the correct format!")
        
        # Check if any of those credentials is None
        if username is None or password is None:
            raise ValueError("Username or password can't be empty!")
        if email is None:
            warnings.warn("Email is not provided, this shit might not work if suspicious login attempt is detected!", UserWarning)

        self.username = username
        self.password = password
        self.email = email

        # For storing all the data
        self.theDict = { "User" : [], "Date" : [], "post_text" : [], "quotedPost_text" : [],
                         "Reply_count": [], "Repost_count": [], "Like_count": [], "View_count": []}
        
        self.login()

    # Utils
    def build_search_url(self, date_limit: str) -> str:
        '''
        This function is used to build the search URL based on the given date limit.
        '''
        return f"{self.SEARCH_URL}{self.FILTERS_COMBINATION}{quote(f' until:{date_limit}')}&f=live&src=typed_query"

    def scrape_detected(self) -> bool:
        try:
            sumthingwrong = WebDriverWait(self.driver, self.WAIT_LONG).until(
                 EC.presence_of_element_located((By.XPATH, '//div[@aria-label="Home timeline"]/div/div/div/span'))
            )
            sumthingwrong.text == "Something went wrong. Try reloading."
            return True
        
        except TimeoutException:
            return False

    def wait_for_posts_or_stepback(self, current_date: str, counter: int) -> tuple[str, int, bool]:
        try:
            # Is there a container for post?
            WebDriverWait(self.driver, self.WAIT_LONG).until(
                EC.presence_of_element_located((By.XPATH, "//div[@data-testid='cellInnerDiv']"))
            )
            counter = 0
            return current_date, counter, False # Yes
        except TimeoutException:
            current_date = minOneDay(current_date)
            counter += 1
            print(f"No posts found, stepping back to {current_date}, attempt {counter + 1}/{self.MAX_EMPTY_PAGES}")
            return current_date, counter, counter > self.MAX_EMPTY_PAGES    # Nothing ever happens and minus by one day

    def parse_post(self, post_element) -> str:
        parts = post_element.find_elements(By.XPATH, ".//span | .//img | .//a[@dir='ltr']")
        text = ""
        for p in parts:
            if p.tag_name == "img":
                text += p.get_attribute("alt")
            elif p.tag_name == "a":
                text += p.text + " "
            else:
                text += p.text
        return text

    def extract_post_data(self, element) -> tuple[str, str, str, str]:
        post_element = element.find_element(
            By.XPATH, './/div[not(@role="link")]/div/div/div/div/div[@data-testid="tweetText"]',
        )
        post_text = self.parse_post(post_element)
        try:
            quoted_element = element.find_element(By.XPATH, './/div[@role="link"]')
            quoted_text = ''.join(
                i.text for i in quoted_element.find_elements(By.XPATH, './/div[@data-testid="tweetText"]/span')
            )
        except NoSuchElementException:
            quoted_text = ""

        post_date = getTime(element.find_element(By.XPATH, './/time').get_attribute("datetime")).strftime("%Y-%m-%d-%H:%M:%S")
        post_user = element.find_element(By.XPATH, './/a/div/span').text
        return post_text, quoted_text, post_user, post_date
    
    def _write_json(self, filename: str) -> None:
        theDict_JSON = {i: {
                j: self.theDict[j][i] for j in self.theDict.keys()
            } for i in range(len(self.theDict["post_text"]))
            }
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(theDict_JSON, f, ensure_ascii=False, indent=4)

    def _write_csv(self, filename: str) -> None:
        df = pd.DataFrame(self.theDict)
        df.to_csv(filename, index=False)

    def _load_latest_savepoint(self) -> bool:
        save_dir = f"Process/{self.processDir}/Savepoints"
        if not os.path.isdir(save_dir):
            return False
        files = [f for f in os.listdir(save_dir) if f.endswith((".csv", ".json"))]
        if not files:
            return False
        latest_file = max(files, key=lambda f: os.path.getmtime(os.path.join(save_dir, f)))
        latest_path = os.path.join(save_dir, latest_file)

        if latest_file.endswith(".csv"):
            df = pd.read_csv(latest_path)
        else:
            with open(latest_path, "r", encoding="utf-8") as f:
                data = json.load(f)
            df = pd.DataFrame.from_dict(data, orient="index")

        self.theDict = {col: df[col].tolist() for col in df.columns}
        if "Date" in self.theDict and self.theDict["Date"]:
            try:
                earliest_dt = min(
                    datetime.strptime(d, "%Y-%m-%d-%H:%M:%S")
                    for d in self.theDict["Date"]
                    if isinstance(d, str)
                )
                self.start_date = earliest_dt.strftime("%Y-%m-%d")
            except Exception:
                pass
        print(f"Resumed from savepoint: {latest_file}")
        return True

    def save(self, type: Literal["final", "savepoint"]) -> str:
        if type not in {"final", "savepoint"}:
            raise ValueError("Save type must be 'final' or 'savepoint'.")

        os.makedirs(f"Process/{self.processDir}/Savepoints", exist_ok=True)
        
        if type == "savepoint":
            save_path = f"Process/{self.processDir}/Savepoints/{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
        else:
            save_path = f"Process/{self.processDir}/Final"

        if self.saveFormat == "csv":
            self._write_csv(f"{save_path}.csv")
        elif self.saveFormat == "json":
            self._write_json(f"{save_path}.json")
        elif self.saveFormat == "both":
            self._write_csv(f"{save_path}.csv")
            self._write_json(f"{save_path}.json")
        else:
            raise ValueError("saveFormat must be 'csv', 'json', or 'both'.")
        
        return save_path
    
    def login(self):
        '''
        Tries to login to the account based from the given credentials

        Will use `self.username` and `self.password` only. However, if X detects your login as suspicious, `self.email` will be used.
        '''
        # Initialize the driver and check bot detection
        self.driver = uc.Chrome()
        self.driver.get('https://www.browserscan.net/bot-detection')

        # Check bot detection, 
        WebDriverWait(self.driver, 30).until(EC.presence_of_element_located((By.XPATH, '//div[@class="_oxrqr1"]')))
        time.sleep(4)
        botResult = self.driver.find_element(By.XPATH, '//strong[@class="_1ikblmd"]').text
        if botResult != "Normal":
            warnings.warn("Bot detection failed! X login might be detected as bot", UserWarning)

        # Get to X login page
        self.driver.get("https://x.com/i/flow/login")
        time.sleep(5)

        # Login handling
        WebDriverWait(self.driver, 30).until(EC.presence_of_element_located((By.XPATH, "//input[@autocomplete = 'username']")))
        self.driver.find_element(By.XPATH, "//input[@autocomplete = 'username']").send_keys(self.username)
        self.driver.find_element(By.XPATH, "//div/button[2]").click()
        time.sleep(5)

        # if email is needed. This means you logged a lot to the account and X raises a suspicious login attempt.
        try:
            if self.driver.find_element(By.XPATH, "//div[1]/div/h1/span/span").text == "Enter your phone number or email address":
                print("Suspicious login attempt detected, attempting to enter email on login prochedures.")
                if self.email is None:
                    raise ValueError("Email is required due to suspicious login attempt, but email is not provided on credentials!")
                self.driver.find_element(By.XPATH, "//input").send_keys(self.email)
                self.driver.find_element(By.XPATH, "//div[2]/div/div/div/button").click()
        except:
            pass

        # Put password
        WebDriverWait(self.driver, 30).until(EC.presence_of_element_located((By.XPATH, "//input[@name='password']")))
        self.driver.find_element(By.XPATH, "//input[@name='password']").send_keys(self.password)
        self.driver.find_element(By.XPATH, "//button[@data-testid='LoginForm_Login_Button']").click()
        time.sleep(5)

        warnings.warn("Please zoom out te browser to 25%, thus there'll be more posts loaded per scroll", UserWarning)
        print("Login sucess!")
        wait(10)

    def start(self, filters, startDate: str = "", endDate: str = "",
              scraping_Params  =  {"wait_short": 10, "wait_long": 30,
                                  "detection_wait": 900, "max_empty_pages": 2},
                                  saveFormat: Literal["csv", "json", "both"] = "csv", 
                                  autoSave: bool = False, autoSaveInterval: int = 15, continue_if_timeout: bool = True,
                                  processDir: str = "", resume_from_savepoint: bool = True):
        
        '''
        This function is used to start the scrapping process based on the given filters.
        
        Parameters
        ----------
        filters : dict
            - A dictionary containing the filters for scrapping.
            - The dictionary should be in the following format:
            ```
            {
                "keywords": "your keywords",
                "from": "username",
                "to": "username",
                "mentions": "username",
                "since": "YYYY-MM-DD",
                "until": "YYYY-MM-DD",
                "min_replies": int,
                "min_likes": int,
                "min_reposts": int,
                "min_views": int,
                "has_links": bool,
                "has_media": bool,
                "has_photos": bool,
                "has_videos": bool,
                "has_gifs": bool
            }
            ```
        '''
        self.SEARCH_URL = "https://x.com/search?q="
        
                # Adjust filters values
        filters["this_exact_phrase"] = f'\"{filters["this_exact_phrase"]}\"' if filters["this_exact_phrase"] != "" else ""

        _any_terms = []
        _any_raw = filters["any_of_these_words"].strip()
        if _any_raw:
            for g1, g2, g3 in re.findall(r'"([^"]+)"|\'([^\']+)\'|(\S+)', _any_raw):
                _any_terms.append(g1 or g2 or g3)
        filters["any_of_these_words"] = (
            f'({" OR ".join(_any_terms)})' if _any_terms else ""
        )

        filters["none_of_these_words"] = f'{" ".join(f"-{i}" for i in filters["none_of_these_words"].split())}' if filters["none_of_these_words"] != "" else ""
        filters["these_hashtags"] = f'({" OR ".join(f"{i}" for i in filters["these_hashtags"].split())})' if filters["these_hashtags"] != "" else ""

        filters["from_accounts"] = f'({" OR ".join(f"from:{i}" for i in filters["from_accounts"].split())})' if filters["from_accounts"] != "" else ""
        filters["to_accounts"] = f'({" OR ".join(f"to:{i}" for i in filters["to_accounts"].split())})' if filters["to_accounts"] != "" else ""
        filters["mentioning_accounts"] = f'({" OR ".join(f"@{i}" for i in filters["mentioning_accounts"].split())})' if filters["mentioning_accounts"] != "" else ""

        filters["replies"] = "" if filters["replies"] else "-filter:replies" 
        filters["links"] = "" if filters["links"] else "-filter:links"

        filters["Minimum_replies"] = f'min_replies:{filters["Minimum_replies"]}' if filters["Minimum_replies"] != "" else ""
        filters["Minimum_likes"] = f'min_faves:{filters["Minimum_likes"]}' if filters["Minimum_likes"] != "" else ""
        filters["Minimum_retweets"] = f'min_retweets:{filters["Minimum_retweets"]}' if filters["Minimum_retweets"] != "" else ""

        FILTERS_COMBINATION = ""
        for _, value in filters.items():
            if value != "":
                FILTERS_COMBINATION += f'{value} '

        self.FILTERS_COMBINATION = quote(FILTERS_COMBINATION.strip())

        # Dates handling
        if startDate == "":
            startDate = datetime.now().strftime("%Y-%m-%d")
        if endDate == "":
            endDate = "2006-01-01"  # Twitter launch date
        self.start_date = startDate
        self.end_date = endDate

        # scraping params
        self.WAIT_SHORT = scraping_Params["wait_short"]
        self.WAIT_LONG = scraping_Params["wait_long"]
        self.DETECTION_WAIT = scraping_Params["detection_wait"]
        self.MAX_EMPTY_PAGES = scraping_Params["max_empty_pages"]

        # Other params
        self.saveFormat = saveFormat
        self.autoSave = autoSave
        self.autoSaveInterval = autoSaveInterval
        self.continue_if_timeout = continue_if_timeout
        self.processDir = processDir if processDir != "" else datetime.now().strftime('%Y-%m-%d')
        
        if resume_from_savepoint:
            self._load_latest_savepoint()
        
        self.scrape()
        
    def scrape(self):
        '''
        This function is used to scrape the posts based on the given filters and date range.
        '''
        reached_all_posts = False
        counter = 0
        seen = set()
        if all(k in self.theDict for k in ("post_text", "Date", "User")) and self.theDict["post_text"]:
            seen = {
                (self.theDict["post_text"][i], self.theDict["Date"][i], self.theDict["User"][i])
                for i in range(len(self.theDict["post_text"]))
            }
        start_date = self.start_date
        try:
            while True:

                # Get the current date upper limit
                current_date_limit = start_date

                # If all shits been scraped, will save and break
                if reached_all_posts:
                    print("All posts have been scraped!")
                    # Delete all temps aka Savepoints
                    shutil.rmtree(f'Process/{self.processDir}/Savepoints/')
                    self.save("final")
                    break

                self.driver.get(self.build_search_url(current_date_limit))
                time.sleep(self.WAIT_SHORT)

                # CHECKER
                ##  1 CHECKER FOR SCRAPING DETECTION, IF `continue_if_timeout` IS TRUE, WILL WAIT AND CONTINUE, ELSE WILL JUST STOP.
                if self.continue_if_timeout:
                    if self.scrape_detected():
                        print("Scraping detected! Auto-saving progress...")
                        self.save("savepoint")
                        print(f"Waiting for {self.DETECTION_WAIT} seconds")
                        # Wait for abyssmal amount of time
                        wait(self.DETECTION_WAIT)
                        continue

                else:
                    if self.scrape_detected():
                        self.save("savepoint")
                        raise RuntimeError("Scraping detected! All progress have been saved.")

                ##  2 CHECKER FOR NO POSTS FOUND, IF SHIT HAPPENS WILL ROLE BACK FOR LIKE A DAY. IF SHIT KEEPS HAPPENING TILL `MAX_EMPTY_PAGES``, WILL STOP.
                start_date, counter, reached_all_posts = self.wait_for_posts_or_stepback(start_date, counter)
                if reached_all_posts:
                    print("No more posts found!")
                    continue

                last_height = self.driver.execute_script("return document.body.scrollHeight")

                while True:
                    elements = self.driver.find_elements(By.XPATH, '//div[@aria-label="Timeline: Search timeline"]/div/div')

                    for element in elements[:-1]:
                        try:
                            post_text, quoted_text, post_user, post_date = self.extract_post_data(element)
                        except (NoSuchElementException, StaleElementReferenceException):
                            continue

                        key = (post_text, post_date, post_user)
                        if key in seen:
                            continue

                        # If end date is reached, functional if user specified end_date at self.start()
                        if self.theDict["Date"] and getTime(self.theDict["Date"][-1]) < getTime(self.end_date):
                            reached_all_posts = True
                            break
                        
                        seen.add(key)
                        self.theDict["post_text"].append(post_text)
                        self.theDict["quotedPost_text"].append(quoted_text)
                        self.theDict["User"].append(post_user)
                        self.theDict["Date"].append(post_date)

                        for group in element.find_elements(By.XPATH, './/div[@role="group"]'):
                            self.theDict["Reply_count"].append(
                                safe_int_from_aria(group.find_element(By.XPATH, './/div[1]/button').get_attribute("aria-label"))
                            )
                            self.theDict["Repost_count"].append(
                                safe_int_from_aria(group.find_element(By.XPATH, './/div[2]/button').get_attribute("aria-label"))
                            )
                            self.theDict["Like_count"].append(
                                safe_int_from_aria(group.find_element(By.XPATH, './/div[3]/button').get_attribute("aria-label"))
                            )
                            self.theDict["View_count"].append(
                                safe_int_from_aria(group.find_element(By.XPATH, './/div[4]/a').get_attribute("aria-label"))
                            )

                        if self.autoSave and len(seen) % self.autoSaveInterval == 0:
                            self.save("savepoint")
                        
                    if reached_all_posts:
                        break
                        
                    self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(self.WAIT_SHORT)
                    new_height = self.driver.execute_script("return document.body.scrollHeight")

                    if new_height == last_height:
                        if self.theDict["Date"]:
                            last_date_only = "-".join(self.theDict["Date"][-1].split("-")[:3])
                            if getTime(last_date_only) >= getTime(current_date_limit):
                                start_date = minOneDay(last_date_only)
                            else:
                                start_date = last_date_only
                        else:
                            start_date = minOneDay(current_date_limit)

                        self.start_date = start_date
                        break

                    last_height = new_height

        except Exception as e:
            print(f"An error occurred: {e}")
            print("Auto-saving progress before exiting...")
            self.save("savepoint")
            self.driver.quit()
            raise e

In [4]:
session = twitterScrapper("Credentials/twitter.json")

In [5]:
FILTERS = {
    # Basic filters
    "all_these_words": "",           # Example: what’s happening · contains both “what’s” and “happening”
    "this_exact_phrase": "",         # Example: what’s happening · contains the exact phrase “what’s happening”
    "any_of_these_words": "\'Makan Bergizi Gratis\' \'MBG\'",        # Example: what’s happening · contains either “what’s” or “happening”
    "none_of_these_words": "grok",       # Example: what’s happening · does not contain the words “what’s” or “happening”
    "these_hashtags": "",            # Example: #whatshappening · contains the hashtag #whatshappening
    
    # Account filters
    "from_accounts": "",          # Example: from:Twitter · Tweets sent from the account Twitter
    "to_accounts": "",            # Example: to:Twitter · Tweets sent in reply to the account Twitter
    "mentioning_accounts": "",    # Example: @Twitter · Tweets that mention the account Twitter

    # Additional filters
    "Minimum_replies": "",         # Example: min_replies:100 · Tweets with at least 100 replies
    "Minimum_likes": "",           # Example: min_faves:100 · Tweets with at least 100 likes
    "Minimum_retweets": "",        # Example: min_retweets:100 · Tweets with at least 100 retweets
    "links": True,                 # Example: filter:links · Include posts with links | If disabled, only posts without links
    "replies": True,               # Example: filter:replies · Include replies and original posts | If disabled, only original posts
}

In [6]:
session.start(
    endDate="2025-12-31",
    filters=FILTERS,
    saveFormat="both",
    autoSave=True,
    autoSaveInterval=100,
    continue_if_timeout=True,
    processDir="MBG",
    resume_from_savepoint=True
)

All posts have been scraped!


# ============================================================================================================================================

## Manual, deprecrated in favour of the syntax above

### Login

In [ ]:
# # 1st fucking option to handle the change

# import zendriver as zd


# browser =  await zd.start()
# await browser.get("https://www.browserscan.net/bot-detection")


In [ ]:
# # 2nd method, which works and also compactivle with selenium function and syntax

# import undetected_chromedriver as uc

# driver = uc.Chrome()

# driver.get('https://www.browserscan.net/bot-detection')

c:\Users\Rhey\anaconda3\envs\scraper\Lib\re\_compiler.py:309: RuntimeWarning: coroutine 'start' was never awaited
  break


In [4]:
with open("Credentials/twitter.json", "r") as f:
    credentials = json.load(f)
    username = credentials["username"]
    password = credentials["password"]
    email = credentials["email"]

# Initialize the driver and check bot detection
driver = uc.Chrome()
driver.get('https://www.browserscan.net/bot-detection')

# Check bot detection, 
WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//div[@class="_oxrqr1"]')))
time.sleep(4)
botResult = driver.find_element(By.XPATH, '//strong[@class="_1ikblmd"]').text
if botResult != "Normal":
    warnings.warn("Bot detection failed! X login might be detected as bot", UserWarning)

# Get to X login page
driver.get("https://x.com/i/flow/login")
time.sleep(5)

# Login handling
WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "//input[@autocomplete = 'username']")))
driver.find_element(By.XPATH, "//input[@autocomplete = 'username']").send_keys(username)
driver.find_element(By.XPATH, "//div/button[2]").click()
time.sleep(5)               # This will wait for the next login pop-up to appear

# if email is neeeded. This means you logged a lot to the account and X raises a suspicious login attempt.
try:
    if driver.find_element(By.XPATH, "//div[1]/div/h1/span/span").text == "Enter your phone number or email address":
        print("Suspicious login attempt detected, attempting to enter email on login prochedures.")
        if email is None:
            raise ValueError("Email is required due to suspicious login attempt, but email is not provided on credentials!")
        driver.find_element(By.XPATH, "//input").send_keys(email)
        driver.find_element(By.XPATH, "//div[2]/div/div/div/button").click()
except:
    pass

# Put password
WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "//input[@name='password']")))
driver.find_element(By.XPATH, "//input[@name='password']").send_keys(password)
driver.find_element(By.XPATH, "//button[@data-testid='LoginForm_Login_Button']").click()
time.sleep(5)

warnings.warn("Please zoom out te browser to 25%, thus there'll be more posts loaded per scroll", UserWarning)
print("Login sucess!")
wait(10)


### Scrapping here

In [5]:
SEARCH_URL = "https://x.com/search?q="

FILTES = {
    # Basic filters
    "all_these_words": "",           # Example: what’s happening · contains both “what’s” and “happening”
    "this_exact_phrase": "",         # Example: what’s happening · contains the exact phrase “what’s happening”
    "any_of_these_words": "\'MBG\' \'Makan Bergizi Gratis\'",        # Example: what’s happening · contains either “what’s” or “happening”
    "none_of_these_words": "",       # Example: what’s happening · does not contain the words “what’s” or “happening”
    "these_hashtags": "",           # Example: #whatshappening · contains the hashtag #whatshappening
    
    # Account filters
    "from_accounts": "",   # Example: from:Twitter · Tweets sent from the account Twitter
    "to_accounts": "",            # Example: to:Twitter · Tweets sent in reply to the account Twitter
    "mentioning_accounts": "",    # Example: @Twitter · Tweets that mention the account Twitter

    # Additional filters
    "Minimum_replies": "",         # Example: min_replies:100 · Tweets with at least 100 replies
    "Minimum_likes": "",           # Example: min_faves:100 · Tweets with at least 100 likes
    "Minimum_retweets": "",        # Example: min_retweets:100 · Tweets with at least 100 retweets
    "links": True,                 # Example: filter:links · Include posts with links | If disabled, only posts without links
    "replies": True,               # Example: filter:replies · Include replies and original posts | If disabled, only original posts
}

# Adjust filters values
FILTES["this_exact_phrase"] = f'\"{FILTES["this_exact_phrase"]}\"' if FILTES["this_exact_phrase"] != "" else ""

_any_terms = []
_any_raw = FILTES["any_of_these_words"].strip()
if _any_raw:
    for g1, g2, g3 in re.findall(r'"([^"]+)"|\'([^\']+)\'|(\S+)', _any_raw):
        _any_terms.append(g1 or g2 or g3)
FILTES["any_of_these_words"] = (
    f'({" OR ".join(_any_terms)})' if _any_terms else ""
 )

FILTES["none_of_these_words"] = f'{" ".join(f"-{i}" for i in FILTES["none_of_these_words"].split())}' if FILTES["none_of_these_words"] != "" else ""
FILTES["these_hashtags"] = f'({" OR ".join(f"{i}" for i in FILTES["these_hashtags"].split())})' if FILTES["these_hashtags"] != "" else ""

FILTES["from_accounts"] = f'({" OR ".join(f"from:{i}" for i in FILTES["from_accounts"].split())})' if FILTES["from_accounts"] != "" else ""
FILTES["to_accounts"] = f'({" OR ".join(f"to:{i}" for i in FILTES["to_accounts"].split())})' if FILTES["to_accounts"] != "" else ""
FILTES["mentioning_accounts"] = f'({" OR ".join(f"@{i}" for i in FILTES["mentioning_accounts"].split())})' if FILTES["mentioning_accounts"] != "" else ""

FILTES["replies"] = "" if FILTES["replies"] else "-filter:replies" 
FILTES["links"] = "" if FILTES["links"] else "-filter:links"

FILTES["Minimum_replies"] = f'min_replies:{FILTES["Minimum_replies"]}' if FILTES["Minimum_replies"] != "" else ""
FILTES["Minimum_likes"] = f'min_faves:{FILTES["Minimum_likes"]}' if FILTES["Minimum_likes"] != "" else ""
FILTES["Minimum_retweets"] = f'min_retweets:{FILTES["Minimum_retweets"]}' if FILTES["Minimum_retweets"] != "" else ""



FILTERS_COMBINATION = ""
for _, value in FILTES.items():
    if value != "":
        FILTERS_COMBINATION += f'{value} '

FILTERS_COMBINATION = quote(FILTERS_COMBINATION.strip())
SEARCH_URL_WITH_FILTERS = f"{SEARCH_URL}{FILTERS_COMBINATION}&f=live&src=typed_query"
SEARCH_URL_WITH_FILTERS

'https://x.com/search?q=%28MBG%20OR%20Makan%20Bergizi%20Gratis%29&f=live&src=typed_query'

In [54]:
theDict = { "User" : [], "Date" : [], "post_text" : [], "quotedPost_text" : [],
            "Reply_count": [], "Repost_count": [], "Like_count": [], "View_count": []}

In [ ]:
# CASE 1, no startdate and enddate given.

timeout = False
reachedallPost = False 
continueifTimeout = True
counter = 0
startDate = datetime.now().strftime("%Y-%m-%d") # Get ur current date

while True:
    currentSearchDate_limit = startDate

    if  reachedallPost:
        print("All posts have been scraped!")
        pd.DataFrame(theDict).to_csv(f'testRhey.csv', index=False)
        break

    # Adjust Link
    searchLink = f"{SEARCH_URL}{FILTERS_COMBINATION}{quote(f" until:{currentSearchDate_limit}")}&f=live&src=typed_query"
    driver.get(searchLink)
    time.sleep(3)
    
    # CHECKERS!
    # This will check if scrapping is detected, if it is. It'll wait for 20 mins
    if continueifTimeout:
        try:
            #   This is essential to check if the scrapping is detected
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//div[@aria-label="Home timeline"]/div/div/div/span[@style="text-overflow: unset;"]')))
            print("Saving!")
            pd.DataFrame(theDict).to_csv(f'Savepoints/{str(datetime.now())}.csv', index=False)
            print("Scrapping detected, waiting for 15 mins")
            wait(900)
            continue
        except:
            pass
    else:
        try:
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//div[@aria-label="Home timeline"]/div/div/div/span[@style="text-overflow: unset;"]')))
            timeout = False
        except:
            pass
        if timeout: break

    # This will wait for the page to load, if nothing exists. Minus one day and repeat
    try:
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//div[@data-testid='cellInnerDiv']")))
    except:
        startDate = minOneDay(startDate)
        if counter == 2:
            print("No more posts found!")
            reachedallPost = True
        counter += 1
        continue
    
    last_height = driver.execute_script("return document.body.scrollHeight")

    # If everything's set, will scrape posts
    while True:

        elements = driver.find_elements(By.XPATH, '//div[@aria-label="Timeline: Search timeline"]/div/div')
        
        for element in elements[:-1]:

            # Get the actual fucking post and its fuckin text an emoji
            try:
                postElement = element.find_element(By.XPATH, './/div[not(@role="link")]/div/div/div/div/div[@data-testid="tweetText"]')
                parts = postElement.find_elements(By.XPATH, ".//span | .//img | .//a[@dir='ltr']")
                postText = ""
                for p in parts:
                    if p.tag_name == "img":
                        postText += p.get_attribute("alt")
                    elif p.tag_name == "a":
                        postText += p.postText + " "
                    else:
                        postText += p.postText
            except:
                # If no text, then continue. Tf ima do with just attachments
                continue

            # Get the fucking quoted post if there is one
            try:
                quotedPostElement = element.find_element(By.XPATH, './/div[@role="link"]')
                quotedPostText = ''.join([i.text for i in quotedPostElement.find_elements(By.XPATH, './/div[@data-testid="tweetText"]/span')])
            except:
                quotedPostText = ""
                pass

            # Check if text exists, if not then continue.
            postDate = getTime(element.find_element(By.XPATH, './/time').get_attribute("datetime")).strftime("%Y-%m-%d-%H:%M:%S")
            postUser = element.find_element(By.XPATH, './/a/div/span').text

            # Check for duplicates, same text, date, and user. If yes, ignore that shit
            if postText in theDict["post_text"] and postDate in theDict["Date"] and postUser in theDict["User"]:
                continue

            # append
            theDict["post_text"].append(postText)
            theDict["quotedPost_text"].append(quotedPostText)
            theDict["User"].append(postUser)
            theDict["Date"].append(postDate)
            
            # post attrs
            for group in element.find_elements(By.XPATH, './/div[@role="group"]'):
                theDict["Reply_count"].append(int(re.search(r'\d+', group.find_element(By.XPATH, './/div[1]/button').get_attribute("aria-label"))[0]))
                theDict["Repost_count"].append(int(re.search(r'\d+', group.find_element(By.XPATH, './/div[2]/button').get_attribute("aria-label"))[0]))
                theDict["Like_count"].append(int(re.search(r'\d+',group.find_element(By.XPATH, './/div[3]/button').get_attribute("aria-label"))[0]))
                theDict["View_count"].append(int(re.search(r'\d+', group.find_element(By.XPATH, './/div[4]/a').get_attribute("aria-label"))[0]) if re.search(r'\d+', group.find_element(By.XPATH, './/div[4]/a').get_attribute("aria-label")) else 0)
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # will break if the scrollbar is at the bottom
        if new_height == last_height:
            
            startDate = '-'.join(theDict["Date"][-1].split("-")[:3])
            # This will check if the untilDate is the same as endDate, if it is. It'll minus one day.
            if currentSearchDate_limit == startDate:
                startDate = minOneDay(startDate)
            
            break

        last_height = new_height

No more posts found!
All posts have been scraped!
